In [1]:
import os

In [2]:
%pwd

'e:\\Projects for portfolio\\Text Summarization\\research'

In [3]:
#Since I want to get into the root directory i.e Text Summarization using Hugging Face API
os.chdir("../")

In [4]:
%pwd

'e:\\Projects for portfolio\\Text Summarization'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig: # defined for the config components present in artifacts for data ingestion
    # Below are the return types for the components (root_dir is Path format , URL is string etc)
    root_dir : Path 
    source_URL : str
    local_data_file : Path
    unzip_dir : Path
    # So basically when we will call this root_dir , we get the value of root_dir from data_ingestion present in config.yaml

# This whole thing updated above is an entity

In [6]:
# Configuration manager
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
    # Here we are reading the yaml file and we can now use the file paths present inside pararms and config.yaml        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) # Here we are calling the artifacts_root key values using '.' , which was the purpose of @ensure_annotations

    def get_data_ingestion_config(self) -> DataIngestionConfig: # Here we are using the entity to specify the return type classes to make sure proper output is returned
        config= self.config.data_ingestion # Calling the data_ingestion dictionary created in config.yaml file

        create_directories([config.root_dir]) # Creating a directory using the root directory

        data_ingestion_config = DataIngestionConfig( # Extracting the values from the config.yaml to here inside data_ingestion_config
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request # To download data from URL
import zipfile # To unzip operation
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig): # It will take the configuration from DataIngestionConfig defined earlier , which will in turn use Configuration Manager to take data from config.yaml
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file): # If file does not exist
            filename, headers = request.urlretrieve( # Download the data from URL
                url= self.config.source_URL, # URL present in config.yaml
                filename = self.config.local_data_file # Path of the file getting saved
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        
        else:
            logger.info(f"File already exists of size : {get_size(Path(self.config.local_data_file))}") # Checking file size present already in the path
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
# Pipeline creation

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config() # Storing the configuration
    data_ingestion = DataIngestion(config=data_ingestion_config) # Using the configuration saved earlier to call data_ingestion
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-29 00:58:39,566: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-29 00:58:39,568: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-29 00:58:39,569: INFO: common: created directory at: artifacts]
[2024-01-29 00:58:39,571: INFO: common: created directory at: artifacts/data_ingestion]
[2024-01-29 00:58:42,984: INFO: 3213381581: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: F030:3528:DFDC6:106B6C:65B69AEF
Accept-Ranges: bytes
Date: Sun, 28 Jan 2024 19:28:57 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10224-MAA
X-Cache: HIT
X